In [1]:
import pandas as pd
import numpy as np

In [2]:
file_path = "processing_data_3.xlsx"
df = pd.read_excel(file_path)

In [3]:
# Nhãn phân loại giá
df["price_level"] = pd.cut(
    df["price"],
    bins=[0, 3_000_000, 10_000_000, 30_000_000, df["price"].max()],
    labels=["Low", "Mid", "High", "Premium"]
)

# Nhãn khuyến mãi
df["discount_status"] = df["has_discount"].apply(lambda x: "Discounted" if x == 1 else "No Discount")

# Nhãn độ phổ biến theo số lượng bán
df["popularity"] = pd.cut(
    df["quantity_sold_value"],
    bins=[-1, 100, 1000, 10_000, df["quantity_sold_value"].max()],
    labels=["Low", "Medium", "High", "Very High"]
)

rating_clean = pd.to_numeric(df["rating_average"], errors="coerce")
rating_clean[(rating_clean < 0) | (rating_clean > 5)] = np.nan

# Nhãn chất lượng theo rating (dựa trên giá trị đã làm sạch)
df["quality_label"] = pd.cut(
    rating_clean,
    bins=[0, 3, 4, 5],
    labels=["Poor", "Good", "Excellent"],
    include_lowest=True
)

# Nhóm thương hiệu
big_brands = ["Apple", "Samsung", "OPPO", "Xiaomi", "Vivo", "Realme"]
df["brand_type"] = df["brand"].apply(lambda x: "Big Brand" if str(x).strip() in big_brands else "Other")

In [4]:
def get_dup_status(row):
    if not row["is_duplicate"]:
        return "Unique"
    elif row["dup_representative"] == "True":
        return "Duplicate Representative"
    else:
        return "Duplicate Copy"

df["dup_status"] = df.apply(get_dup_status, axis=1)

In [5]:
df.to_excel("processing_data_4.xlsx", index=False)
print("[INFO] Đã lưu processing_data_4.xlsx với các cột enriched mới")

[INFO] Đã lưu processing_data_4.xlsx với các cột enriched mới
